In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/austinLorenzMccoy/CompleteDSproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="austinLorenzMccoy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1d06b3f1dc94bb2bb3ed0960c7d406847b9d362d"

In [2]:
%pwd

'/Users/a/Documents/DataScience_World/ML10_end_to_end/dsproject/CompleteDSproject/research'

In [3]:
import os
os.chdir("../")
%pwd

'/Users/a/Documents/DataScience_World/ML10_end_to_end/dsproject/CompleteDSproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_tracking_uri: str  # Ensure this matches exactly


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
@dataclass
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = Path("config/config.yaml"),
        params_filepath: Path = Path("params.yaml"),
        schema_filepath: Path = Path("schema.yaml")
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        model_params = self.params.KerasModel  
        target_column = 'target'  

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(self.config.data_transformation.transformed_data_path),
            model_path=Path(config.model_path),
            metric_file_name=Path(config.metric_file_name),
            all_params=model_params,
            target_column=target_column,
            mlflow_tracking_uri="https://dagshub.com/austinLorenzMccoy/CompleteDSproject.mlflow"  # Make sure this matches the attribute name
        )
        return model_evaluation_config



In [7]:
import os
import pandas as pd
import numpy as np
import joblib
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn


In [8]:
from sklearn.metrics import precision_score, accuracy_score,recall_score, f1_score
import pandas as pd
import mlflow
from pathlib import Path
import joblib
from urllib.parse import urlparse
import mlflow.sklearn  # Instead of mlflow.keras
from src.datascience.utils.common import create_directories, read_yaml, save_json
from src.datascience.entity.config_entity import ModelEvaluationConfig
import os

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted')
        recall = recall_score(actual, pred, average='weighted')
        f1 = f1_score(actual, pred, average='weighted')
        return accuracy, precision, recall, f1

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(['target'], axis=1)
        test_y = test_data['target']

        mlflow.set_registry_uri(self.config.mlflow_uri)  # Changed from mlflow_tracking_uri
       
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (accuracy, precision, recall, f1) = self.eval_metrics(test_y, predicted_qualities)

            scores = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

            # Save metrics locally
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1", f1)

            # Log model to MLflow
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="SklearnModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-09 12:57:02,358: INFO: common: YAML file loaded successfully from: config/config.yaml]
[2025-01-09 12:57:02,362: INFO: common: YAML file loaded successfully from: params.yaml]
[2025-01-09 12:57:02,365: INFO: common: YAML file loaded successfully from: schema.yaml]
[2025-01-09 12:57:02,366: INFO: common: Created directory at: artifacts]
[2025-01-09 12:57:02,368: INFO: common: Created directory at: artifacts/model_evaluation]


TypeError: ModelEvaluationConfig.__init__() got an unexpected keyword argument 'mlflow_tracking_uri'